# 2A.ml - Pipeline pour un réduction d'une forêt aléatoire - énoncé

Le modèle Lasso permet de sélectionner des variables, une forêt aléatoire produit une prédiction comme étant la moyenne d'arbres de régression. Cet aspect a été abordé dans le notebook [Reduction d'une forêt aléatoire](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/td2a_tree_selection_correction.html). On cherche à automatiser le processus.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline

## Datasets

Comme il faut toujours des données, on prend ce jeu [Diabetes](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).

In [3]:
from sklearn.datasets import load_diabetes
data = load_diabetes()
X, y = data.data, data.target

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Forêt aléatoire suivi de Lasso

La méthode consiste à apprendre une forêt aléatoire puis à effectuer d'une régression sur chacun des estimateurs.

In [5]:
import numpy
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

# Apprentissage d'une forêt aléatoire
clr = RandomForestRegressor()
clr.fit(X_train, y_train)

# Récupération de la prédiction de chaque arbre
X_train_2 = numpy.zeros((X_train.shape[0], len(clr.estimators_)))
estimators = numpy.array(clr.estimators_).ravel()
for i, est in enumerate(estimators):
    pred = est.predict(X_train)
    X_train_2[:, i] = pred

# Apprentissage d'une régression Lasso
lrs = Lasso(max_iter=10000)
lrs.fit(X_train_2, y_train)
lrs.coef_

array([ 0.01058919,  0.05879275, -0.00490468,  0.0422317 ,  0.02061981,
        0.05832323,  0.04902792, -0.02386671, -0.00783027, -0.02905091,
       -0.05936758, -0.03081102, -0.00874234, -0.01032493, -0.00215755,
        0.02104254, -0.06726193,  0.00863015, -0.00657562,  0.01915455,
        0.1103515 ,  0.03127041,  0.0059957 ,  0.01318572, -0.02425179,
        0.02444136, -0.01270415,  0.00860503, -0.01053657, -0.0044742 ,
       -0.01316523,  0.01369104, -0.00739582, -0.02240202, -0.0049985 ,
        0.08646501,  0.00866649, -0.00228254,  0.02181667,  0.01934537,
       -0.00796704, -0.00372213,  0.02581304, -0.01812068,  0.04921884,
        0.04735237, -0.01544872,  0.00383606,  0.03220245,  0.04162666,
        0.00815848,  0.04327313,  0.03816147, -0.00254619,  0.        ,
       -0.03287036, -0.04364327,  0.00691009, -0.00819448,  0.00571863,
       -0.0085195 ,  0.03282482, -0.041993  ,  0.04787454,  0.01832266,
        0.03145652,  0.013905  ,  0.00592087,  0.01296335,  0.01

Nous avons réussi à reproduire le processus dans son ensemble. Pas toujours simple de se souvenir de toutes les étapes, c'est pourquoi il est plus simple de compiler l'ensemble dans un [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

## Exercice 1 : Premier pipeline

Peut-être trouverez-vous tout de suite un [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) qui fonctionne. La partie difficile est la partie qui produit le vecteur des sorties de chaque arbre de régression. La première piste que j'ai explorée est un [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html).

## Exercice 2 : Second pipeline

La première idée de marche pas vraiment... On décide alors de déguiser la forêt aléatoire en un transformeur.

In [7]:
class RandomForestRegressorAsTransformer:
    
    def __init__(self, **kwargs):
        self.rf = RandomForestRegressor(**kwargs)
        
    def fit(self, X, y):
        # ...
        return self
        
    def transform(self, X):
        # ...
        # return les prédiction de chaque arbre
        pass

# Tout ça pour écrire ce qui suit...
trrf = RandomForestRegressorAsTransformer()
trrf.fit(X_train, y_train)
trrf.transform(X_train)

Il reste à écrire le pipeline correspondant à la séquence d'apprentissage décrit quelque part dans ce notebook.

In [8]:
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[
    ('name', 'passthrough'),
    # ...
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('name', 'passthrough')])

## Exercice 3 : GridSearchCV

Comme l'ensemble des traitements sont maintenant dans un seul pipeline que *scikit-learn* considère comme un modèle comme les autres, on peut rechercher les meilleurs hyper-paramètres du modèle, comme le nombre d'arbres initial, le paramètre *alpha*, la profondeur des arbres... Tout ça avec la classe [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Vous devriez tomber sur un message disant que la classe ``RandomForestRegressorAsTransformer`` a besoin de la méthode *set_params*... Un indice : ``def set_params(self, **params): self.rf.set_params(**params)``.

## Exercice 4 : nombre de coefficients non nuls

Il ne reste plus qu'à trouver le nombre de coefficients non nuls du meilleur modèle, donc le nombre d'arbres conservés par le modèle.